# Getting Started

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AustralianCancerDataNetwork/pydicer/blob/main/examples/GettingStarted.ipynb)

This notebook provides a basic example to run the PyDicer pipeline using some test data.

In [ ]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path

from pydicer.input.test import TestInput

## Setup working directory

First we'll create a directory for our project. Change the `directory` location to a folder on your
system where you'd like PyDicer to work with this data.

In [ ]:
directory = Path("./data")

## Create a PyDicer object

The [PyDicer class](https://australiancancerdatanetwork.github.io/pydicer/tool.html) provides all functionlity to run the pipeline and work with the data stored and
converted in your project directory

In [ ]:
pydicer = PyDicer(directory)

## Fetch some data

A [TestInput class](https://australiancancerdatanetwork.github.io/pydicer/input.html#pydicer.input.test.TestInput) is provided in pydicer to download some sample data to work with. Several other
input classes exist if you'd like to retrieve DICOM data for conversion from somewhere else, [see 
the docs for information on how these work](https://australiancancerdatanetwork.github.io/pydicer/html/input.html).

In [ ]:
dicom_directory = directory.joinpath("dicom")
test_input = TestInput(dicom_directory)
test_input.fetch_data()

# Add the input DICOM location to the pydicer object
pydicer.add_input(dicom_directory)

## Run the pipeline

The function runs the entire PyDicer pipeline on the test DICOM data. This includes:
- [Preprocessing](https://australiancancerdatanetwork.github.io/pydicer/preprocess.html) the DICOM data (data which can't be handled or is corrupt will be placed in Quarantine)
- [Convert](https://australiancancerdatanetwork.github.io/pydicer/convert.html) the data to Nifti format (see the output in the `data` directory)
- [Visualise](https://australiancancerdatanetwork.github.io/pydicer/visualise.html) the data (png files will be placed alongside the converted Nifti files)
- [Compute Radiomics features](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.compute_radiomics) (Results are stored in a csv alongside the converted structures)
- [Compute Dose Volume Histograms](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.compute_dvh) (results are stored alongside converted dose data)

> Note that the entire Pipeline can be quite time consuming to run. Depending on your project's
> dataset you will likely want to run only portions of the pipeline with finer control over each
> step. For this reason we only run the pipeline for one patient here as a demonstration.

In [ ]:
pydicer.run_pipeline(patient="HNSCC-01-0019")

## Prepare a dataset

Datasets which are extracted in DICOM format can often be a bit messy and require some cleaning up
after conversion. Exactly what data objects to extract for the clean dataset will differ by project
but here we use a somewhat common approach of extracting the latest structure set for each patient
and the image linked to that.

The resulting dataset is stored in a folder with your dataset name (`clean` for this example).

See the [dataset preparation example](https://australiancancerdatanetwork.github.io/pydicer/_examples/DatasetPreparation.html) for a more detailed description on how this works.


In [ ]:
pydicer.dataset.prepare(dataset_name="clean", preparation_function="rt_latest_dose")

## Analyse the dataset

The pipeline computes first-order radiomics features by default, as well as dose volume histograms.
Here we can extract out the results easily into a Pandas DataFrame for analysis.

Check out the [Compute Radiomics](https://australiancancerdatanetwork.github.io/pydicer/_examples/Radiomics.html) and the [Dose Metrics](https://australiancancerdatanetwork.github.io/pydicer/_examples/DoseMetrics.html) examples for further details on how to use these functions.

In [ ]:
# Display the DataFrame of radiomics computed
df_radiomics = pydicer.analyse.get_all_computed_radiomics_for_dataset(dataset_name="clean")
df_radiomics

In [ ]:
# Extract the D95, D50 and V3 dose metrics
df_dose_metrics = pydicer.analyse.compute_dose_metrics(dataset_name="clean", d_point=[95, 50], v_point=[3])
df_dose_metrics